# SPE Acceptance #

This notebook runs most of the code needed to do SPE acceptance studies. It starts by making runlists that are used to group the different types of SPE runs together. These runlists are used to download the rawdata if it is unavailable on midway, and then submit midway jobs to process the raw data in way needed for this study. After the jobs finish (~6 hours), the data is read in and plotted in various ways. 

See the python modules in this repository for more details, especially spe_acceptance.py and analyze.py

In [1]:
import matplotlib
matplotlib.rc('font', size=16)
import matplotlib.pyplot as plt
import analyze
from channel_dict import channel_dict
import numpy as np
from matplotlib import cm
import os
import pymongo 
import re
from tqdm import tqdm
from pax.configuration import load_configuration
import hax
from hax.pmt_plot import plot_on_pmt_arrays, pmt_data
import pandas as pd

#### Make runlists for any new runs that have been taken ####

In [ ]:
from make_runlist_new import write_spe_lists
# dry run
write_spe_lists(write=False)

In [ ]:
# if happy with dry run, write the files
wrote = write_spe_lists(write=True)
print("We wrote these files:")
for f in wrote:
    print('\t' + f)

In [ ]:
with open("tmp_submit_file.txt", "w") as f:
    for file in wrote:
        f.write("%s\n" % file)

In [ ]:
%%bash

./notebook_submit.sh


#### Check if jobs still running ####

In [ ]:
from subprocess import Popen, PIPE
import time

def jobs_running():
    output = Popen(["squeue","--user", os.environ['USER']], stdin=PIPE, stdout=PIPE, stderr=PIPE).stdout.read()
    output = output.decode("utf-8").split("\n")
    jobs = len([l for l in output if 'spe' in l])
    return (jobs > 0)

# wait for jobs to finish
print("waiting for jobs to finish")
while jobs_running():
    print("..", end='')
    time.sleep(60)
print("\nDONE")

# Use the same runlist files to read in the data #

In [ ]:
uri = 'mongodb://eb:%s@xenon1t-daq.lngs.infn.it:27017,copslx50.fysik.su.se:27017,zenigata.uchicago.edu:270\
17/run'
uri = uri % os.environ.get('MONGO_PASSWORD')
c = pymongo.MongoClient(uri,
                        replicaSet='runs',
                        readPreference='secondaryPreferred')
db = c['run']
collection = db['runs_new']

# get runlist files so that we know which runs go together
def get_runlist_files(dir, exclude = []):
    runlist_files = ['%s/%s' % (dir, f) for f in os.listdir(dir) if f.startswith("runlist") and f not in exclude and f.endswith(".txt")]
    return runlist_files

# find a regular run from which to extract self trigger thresholds
def find_regular_run(LED_run):
    query = {'source.type' : {'$ne' : 'LED'},
             '$and' : [{'number' : {'$lt': LED_run+20}}, 
                       {'number' : {'$gt' : LED_run-20}}
                      ]
            }
    cursor = collection.find(query, {'number' : True,
                                     'reader' : True,
                                     '_id' : False})
    
    
    
    runs = np.array([run['number'] for run in cursor 
                     if any([ r['register'] == '8060' 
                             for r in run['reader']['ini']['registers']])])
    
    if LED_run < 5144: #when thresholds changed
        runs = runs[runs<5144]
    elif LED_run > 5144:
        runs = runs[runs>5144]
    diff = abs(runs - LED_run)
    
    closest_run = runs[np.argmin(diff)]
    
    return closest_run
        
    
def get_threshold_changes():
    cursor = collection.find(
        {
            "tags.name": "_sciencerun0", 
            #"source.type": "AmBe",
            #"data": {"$elemMatch": {"type": "processed", "pax_version": "v6.4.2", "host": "midway-login1"}}
        }).sort("number", 1)


    # Loop
    st_list = []

    st_changes = []
    for doc in cursor:

        # Record self-trigger thresholds
        checklist = []
        for register in doc['reader']['ini']['registers']:
            if register['register'][-2:] == '60':
                checklist.append(register)
        sortedcheck = sorted(checklist, key=lambda k: k['register'])

        if sortedcheck != st_list:
            #print("Registers changed run " + str(doc['number']))
            st_changes.append(doc['number'])
            st_list = sortedcheck
    return st_changes

In [ ]:
runlist_files = get_runlist_files('/home/ershockley/analysis/SPE/runlists',
                                  exclude = [])

In [ ]:
means = []
medians = []
stds = []
bot_runs = []

n_bins = 40

acc_array = np.ones((len(runlist_files), len(channel_dict['all_channels'])))


for file, runlist in enumerate(sorted(runlist_files)):
    runlist = runlist.split('/')[-1].split('_')
    bottom_run = int(runlist[1])
    topbulk_run = int(runlist[2])
    topring_run = int(runlist[3][:-4])
    csv_file = "./acceptance_data/acceptances_%d_%d_%d.csv" % (bottom_run, topbulk_run, topring_run)
    
    #if os.path.exists(csv_file):
    #    continue
        
        
    existing_files = [os.path.exists('/home/ershockley/analysis/SPE/data/run_%d' % r)
                        for r in [bottom_run, topbulk_run, topring_run]]
    if not all([os.path.exists('/home/ershockley/analysis/SPE/data/run_%d' % r)
                for r in [bottom_run, topbulk_run, topring_run]]):
        missing_runs = [str(run) for run, boo in zip([bottom_run, topbulk_run, topring_run],
                                               existing_files) if not boo]
        print("Missing data for runs %s" % ",".join(missing_runs))
        continue
    
    threshold_run = find_regular_run(bottom_run)
    print("Threshold run: %d" % threshold_run)
    try:
        thresholds = analyze.get_thresholds(threshold_run)
    except KeyError:
        thresholds = analyze.get_thresholds(threshold_run + 1)
        
    acceptances = analyze.get_acceptances_3runs(bottom_run, topring_run, 
                                                topbulk_run, thresholds, plot=True)
    
    
    acc_array[file, :] *= acceptances
    

    with open(csv_file, "w") as f:
        f.write("channel,acceptance\n")
        for ch, acc in enumerate(acceptances):
            f.write("%d,%0.4f\n" % (ch, acc))
        
    on_accs = acceptances
    on_accs = np.delete(on_accs, analyze.excluded_pmts)
    
    means.append(np.mean(on_accs))
    medians.append(np.median(on_accs))
    stds.append(np.std(on_accs))
    bot_runs.append(bottom_run)
    plt.figure()
    acc_hist, bins, patches = plt.hist(acceptances, bins = n_bins, range = (0,1))
    plt.title("Runs %d %d %d" % (bottom_run, topbulk_run, topring_run))
    plt.xlabel("SPE acceptance fraction")
    plt.ylabel(" # channels / bin (%d bins)" % n_bins)
    plt.show()
    #plt.savefig("/project/lgrandi/xenon1t/spe_acceptance/plots/hist_%d-%d-%d.png" % 
    #        (bottom_run, topbulk_run, topring_run))
    
print('DONE')


In [ ]:
#plot all occupancies as a function of run number
def get_occ(c):
    return -1*np.log(c)

run_numbers=[]

all_run_numbers=[]

bottom_runs=[]
topb_runs=[]

topring_runs=[]

start_run = 6731
end_run = 1e6


datadir = 'acceptance_data'
acceptance_dict = {}

#adjusted for higher occs
for file in os.listdir(datadir):
    runs=file.split('_')
    bottom_run = int(runs[1])
    topb_run=int(runs[2])
    topring_run=int(runs[3].split('.')[0])
    
    if not (start_run < bottom_run < end_run):
         continue 
            
#    if topb_run!=topring_run:
#        continue
    else:
        bottom_runs.append(bottom_run)
        topb_runs.append(topb_run)
        topring_runs.append(topring_run)
        all_run_numbers.append(bottom_run)
        #all_run_numbers.append(topb_run)
        all_run_numbers.append(topring_run)    
    
    data = pd.read_csv(datadir+'/'+file)
    
    acceptance_dict[bottom_run] = data['acceptance'].values 

for file, runlist in enumerate(sorted(runlist_files)):
    runlist = runlist.split('/')[-1].split('.')[0].split('_')[1:]
    for number in runlist:            
        run_numbers.append(number)
        
run_numbers = [int(r) for r in run_numbers]
all_run_numbers=sorted(all_run_numbers)

ch, corr = analyze.get_corrections(run_numbers[0])
plt.hist(get_occ(corr), bins=20)
plt.show()


In [ ]:
#all runs
corrections=np.ones((len(all_run_numbers), 248))
occs = np.ones_like(corrections)
occ_df = {}

for i, run in enumerate(all_run_numbers):
    ch, corr=analyze.get_corrections(run)
    #corrections[i] = corr
    #occs[i] = get_occ(corr)
    occ_df[run] = get_occ(corr)
    


In [ ]:
occ_df = pd.DataFrame(occ_df)


occ_df.head()


In [ ]:
skip_runs = []
for run, occ in zip(tmp_bot_runs, bottom_occs):
    if occ < 0.1:
        print('low', run)
    elif occ > 0.5:
        print('high', run)
    elif 0.1 < occ< 0.36:
        print('slightly low', run)
        
    if (occ<0.36) or (occ>0.5):
        skip_runs.append(run)
        
skip_runs = np.array(skip_runs)

# Plot SPE acceptance as function of time #

In [ ]:
print(sorted(bot_runs))

In [ ]:
# read in data files

evo_data = pd.DataFrame(acceptance_dict)

#bad_bot_runs=[13837, 13433, 14088]
#for i in bad_bot_runs:
#    del evo_data[i]

#evo_data = evo_data.drop(12512)
#evo_data = evo_data.drop(skip_runs, axis=1)
evo_data.head()

    

In [ ]:
xx, yy = np.meshgrid(evo_data.columns, evo_data.index)

plt.figure(figsize=(16,12))
plt.pcolor(xx, yy, evo_data.values, cmap='viridis_r', vmin=0.5, vmax=1.0)
plt.title("SPE acceptance evolution per channel")
plt.xlabel("Run number")
plt.ylabel("Channel")
plt.colorbar()
plt.show()

In [ ]:
neg_channels = []
for ch, row in evo_data.iterrows():
    if min(row.tolist()) < 0:
         neg_channels.append(ch)
print(neg_channels)

In [ ]:
# can now look at acceptace for individual run

from matplotlib.ticker import AutoMinorLocator
from matplotlib.dates import DayLocator, MonthLocator, DateFormatter, drange, AutoDateFormatter, AutoDateLocator
from datetime import datetime

def plot_channel_evo(channel, dataframe, vlines=None):
    plt.figure(figsize=(10, 6))
    #times = [get_run_time(run) for run in dataframe.columns]
    plt.scatter(dataframe.columns, dataframe.loc[[channel]])
    plt.title('Channel %d' % channel)
    plt.ylabel('SPE acceptance fraction')
    plt.xlabel('Run number')
    plt.ylim(-0.05, 1.1)
    ax = plt.gca()
    ax.yaxis.grid(True) #(b=True, which='major', color='0.65',linestyle='-')
#    ax.xaxis.set_major_locator(MonthLocator())
#    ax.xaxis.set_minor_locator(DayLocator())
#    locator = AutoDateLocator()
#    ax.xaxis.set_major_formatter(DateFormatter("%b '%y"))
    if vlines is not None:
        n_regions = len(vlines) + 1
        regions = []
        for line in vlines:
            regions.append(r for r in dataframe.columns if r < line)
            print(np.mean(dataframe[regions].loc[[channel]]))
#            plt.axvline(get_run_time(line))
            plt.axvline(line)

## plot average acceptance vs run number ##

In [ ]:
# ignore off channels (acceptance < 0.05)

def custom_mean(df):
    new_means = []
    for col in sorted(df):
        new_vals = [val for val in df[col] if val > 0.05]
        new_means.append(np.mean(new_vals))
    d = {}
    for i, mean in enumerate(new_means):
        d[df.columns[i]] = mean
    return pd.DataFrame(d, index=['mean'])

def custom_median(df):
    new_meds = []
    for col in sorted(df):
        new_vals = [val for val in df[col] if val > 0.05]
        new_meds.append(np.median(new_vals))
    d = {}
    for i, med in enumerate(new_meds):
        d[df.columns[i]] = med
    return pd.DataFrame(d, index=['median'])

def custom_quantile(df, q):
    new_qs = []
    for col in sorted(df):
        new_vals = [val for val in df[col] if val > 0.05]
        new_qs.append(np.percentile(new_vals, q))
    d = {}
    for i, val in enumerate(new_qs):
        d[df.columns[i]] = val
    return pd.DataFrame(d, index=['%d_percentile'%q])

    
    

## Fluctuations?

In [ ]:
def group_list(runlist):
    runlist = np.array(sorted(runlist))
    indices  = np.where(runlist[:-1] - runlist[1:] != -1)[0]
    starts = runlist[np.insert(indices + 1, [0], 0)]
    ends = runlist[np.append(indices, [-1])]
    return [(start, end) for start, end in zip(starts, ends)]

def run_ranges(source, **kwargs):
    runlist = runs_by_source(source, **kwargs)
    return group_list(runlist)

In [ ]:
from runDB import runs_by_source
kr = runs_by_source('Kr83m', num_range=(6000, 16000))
rn = runs_by_source('Rn220', num_range=(6000, 16000))
bkg = runs_by_source('none', num_range=(6000, 16000))
kr_ranges = group_list(kr)
rn_ranges = group_list(rn)
bkg_ranges = group_list(bkg)

In [ ]:
import hax
hax.init()

def get_run_time(run):
    return hax.runs.datasets[hax.runs.datasets.number == run].start.values[0]

In [ ]:
dates = [get_run_time(run) for run in evo_data.columns]

## Noise evolution

In [ ]:
noise_runs = []
noise_rms = []
noise_errors = [[],[]]

with open('/home/ershockley/analysis/SPE/noise_rms.csv') as f:
    for num, line in enumerate(f.readlines()):
        if num==0:
            continue
        line = line.split(',')
        run, rms, lower, upper = int(line[0]), float(line[1]), float(line[2]), float(line[3])
        noise_runs.append(run)
        noise_rms.append(rms)
        noise_errors[0].append(lower)
        noise_errors[1].append(upper)
        
noise_dates = [get_run_time(run) for run in noise_runs]

In [ ]:
print(sorted(bottom_runs))

In [ ]:
bad_runs = [6169, 10877, 11069, 12211, 12511, 12768, 13837]

In [ ]:
for col in evo_data.coli,ms

In [ ]:
import matplotlib.dates as mdates
years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
fmt = mdates.DateFormatter('%Y-%m')

thresh_change = get_run_time(6845)

median = custom_median(evo_data).values[0]
lower = custom_quantile(evo_data, 16).values[0]
upper = custom_quantile(evo_data, 84).values[0]
lower = median-lower
upper = upper-median

cut_range = (0.93, 0.96)
f, ax = plt.subplots(figsize=(14,10))
#ax.scatter(dates, custom_mean(evo_data).values[0],
#           label = 'average SPE acceptance', color='navy')

ax.errorbar(dates, median,
            label='SPE acceptance', color='black', linestyle='None', 
            marker='.')
ax.axvline(thresh_change, color='navy')
ax.text(0.06, 0.9, 'threshold change', transform=ax.transAxes, color='navy')
ax.set_ylabel('SPE acceptance')
ax.set_xlabel('Date')
ax.grid() 
ax.set_ylim(0.75, 1.05)

plt.xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=20)
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(fmt)

ax2 = ax.twinx()
#ax2.scatter(dates, noise_sigmas, color='red', s=40)
ax2.errorbar(noise_dates, noise_rms,
             color='red', linestyle='None', marker='.')
ax2.set_ylabel('noise RMS [ADC counts]', color='red')
ax2.tick_params('y', colors='red')
ax2.set_ylim(1, 6)
#ax2.legend(loc=(0.05, 0.82))
plt.savefig('time_evolution_noise.png')
plt.show()

## Thresholds 

In [ ]:
changes = []
thresholds = []
previous_threshold=0
for run in sorted(bottom_runs):
    regrun = find_regular_run(run)
    thresh = analyze.get_thresholds(regrun)
    thresholds.append(thresh)
    mean_threshold = np.mean(thresh)
    print(run, regrun, mean_threshold)
    if mean_threshold != previous_threshold:
        changes.append(run)
    previous_threshold=mean_threshold

In [ ]:
np.mean(thresholds[2][:247])

In [ ]:
old_thresholds = thresholds[0]
new_thresholds = thresholds[2]

with open('thresholds.txt', 'w') as f:
    header = '^channel^before run 6893^after run 6893^'
    print(header)
    f.write(header + '\n')
    for ch, (old, new) in enumerate(zip(old_thresholds, new_thresholds)):
        if ch > 247:
            continue
        line = '|%d|%d|%d|' % (ch, old, new)
        print(line)
        f.write(line + '\n')

In [ ]:
changed_channels = {}
for run in changes:
    print(run)
    index = sorted(bottom_runs).index(run)
    if index==0:
        print('skipping first run')
        continue
    prev_run = sorted(bottom_runs)[index - 1]
    thresh = thresholds[index]
    prev_thresh = thresholds[index-1]
    changed_channels[run] = np.where(thresh-prev_thresh != 0)
    
print(changed_channels)

## Acceptance for each channel

In [ ]:
evo_data.head()

In [ ]:
regions = [(0,6844), (6845, 99999)]
per_ch_means = []

for r in regions:
    tmp = evo_data.drop(evo_data.columns[evo_data.columns < r[0]], axis=1)
    tmp = tmp.drop(tmp.columns[tmp.columns > r[1]], axis=1)
    tmp['mean'] = tmp.apply(np.mean, axis=1)
    per_ch_means.append(tmp['mean'].values)

In [ ]:
per_ch_means = np.array(per_ch_means)
print('^ch^acceptance before run 6893^acceptance after run 6893')
for ch in range(248):
    before, after = per_ch_means[:,ch][0], per_ch_means[:,ch][1]
    print('|%d|%0.2f|%0.2f|'%(ch, before, after))

In [ ]:
print('mean before: ',np.mean([val for val in per_ch_means[0] if val > 0.05]))
print('mean after: ',np.mean([val for val in per_ch_means[1] if val > 0.05]))

print('median before: ',np.median([val for val in per_ch_means[0] if val > 0.05]))
print('median after: ',np.median([val for val in per_ch_means[1] if val > 0.05]))

## Runs where we saw changes in low acceptance pmts ## 

In [ ]:
# retrieve low acceptance pmts

low_acc_dict = {}

for col in evo_data:
    low_acc_dict[col] = [ch for ch, a in enumerate(evo_data[col]) if a < 0.5]

In [ ]:
low_acc_pmts = []
for run, l in low_acc_dict.items():
    low_acc_pmts = list(set(low_acc_pmts).union(set(l)))
    
print(sorted(low_acc_pmts))

In [ ]:
last_list = 'init'
changes = {}
for key in sorted(low_acc_dict):
    this_list = sorted(low_acc_dict[key])
    if last_list == 'init':
        last_list = this_list
        
    if this_list != last_list:
        changed_pmts = set(last_list).symmetric_difference(set(this_list))
        changes[key] = changed_pmts
        
    last_list = this_list

In [ ]:
s = "Runs where we saw changes in low acceptance PMTS, and the PMTS that changed"
print(s)
print("-" * len(s))

for run in sorted(changes):
    print(run, changes[run])

In [ ]:
# plot the evolution of the pmts that changed:
pmts_that_changed = []
for run in changes:
    for ch in changes[run]:
        pmts_that_changed.append(ch)

for ch in pmts_that_changed:
    plot_channel_evo(ch, evo_data)
    plt.show()

In [ ]:
plot_channel_evo(137, evo_data)
dplt.show()